In [1]:
import os
from backoff import on_exception, expo
import json
import requests
from collections import Counter
from datetime import datetime
from ratelimit import limits, sleep_and_retry, RateLimitException
import time
import urllib.request
import asyncio
import aiohttp
import nest_asyncio
from async_timeout import timeout
from requests.exceptions import HTTPError

In [2]:
data_path = "D:\\METIS\\data\\"
image_path = "D:\\METIS\\images\\"

images_saved = []
met_objects = []

In [3]:
url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects'
response = requests.get(url, allow_redirects=True)
all_objects = json.loads(response.text)
objectIDs = all_objects['objectIDs']

In [4]:
@sleep_and_retry
@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_object_details(object_id, img_url=True):
    url ='https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'.format(object_id)
    response = requests.get(url, timeout=10)
    response_json = json.loads(response.text)
    met_objects.append(response_json)
    
    if img_url:
        return response_json['primaryImage']

@on_exception(expo, RateLimitException, max_tries=8)
@limits(calls=300, period=300)
def get_image(object_id, url):
    filename = image_path + "{}.jpg".format(object_id)
    im = urllib.request.urlretrieve(url, filename)
    images_saved.append({
                        'objectID': object_id, 
                        'url': url,
                        'filename': filename
                        })

In [5]:
img_url = get_object_details(1567)
get_image(1567, img_url)

In [6]:
images_saved

[{'objectID': 1567,
  'url': 'https://images.metmuseum.org/CRDImages/ad/original/79684.jpg',
  'filename': 'D:\\METIS\\images\\1567.jpg'}]

In [7]:
images_saved = []
met_objects = []

start = datetime.now()

for i, object_id in enumerate(objectIDs):
    if i % 1000 == 0:
        with open(data_path + "images_saved.json", "w") as write_file:
            json.dump(images_saved, write_file)   
            
        with open(data_path + "met_objects.json", "w") as write_file:
            json.dump(met_objects, write_file)  
            
        print("{}: {} elapsed".format(i, datetime.now() - start))
            
    try:
        img_url = get_object_details(object_id)
        get_image(object_id, img_url)
        
    except:
        pass
   # print(i, object_id)

In [8]:
start = datetime.now()

for i, object_id in enumerate(objectIDs[400000:420000]):
    if i % 1000 == 0:
        with open(data_path + "images_saved_400.json", "w") as write_file:
            json.dump(images_saved, write_file)   
            
        with open(data_path + "met_objects_400.json", "w") as write_file:
            json.dump(met_objects, write_file)  
            
        print("{}: {} elapsed".format(i + 400000, datetime.now() - start))
            
    try:
        img_url = get_object_details(object_id)
        get_image(object_id, img_url)
        
    except:
        pass

400000: 0:00:00.104993 elapsed
401000: 0:15:49.974967 elapsed
402000: 0:31:45.109865 elapsed
403000: 0:50:36.067833 elapsed
404000: 1:07:03.944354 elapsed
405000: 1:22:15.368547 elapsed
406000: 1:40:48.083772 elapsed
407000: 1:59:04.999368 elapsed
408000: 2:16:39.463737 elapsed
409000: 2:34:55.938614 elapsed
410000: 2:51:59.603581 elapsed
411000: 3:09:21.512161 elapsed
412000: 3:25:12.088697 elapsed
413000: 3:43:24.586210 elapsed
414000: 4:00:32.985261 elapsed
415000: 4:17:43.899093 elapsed
416000: 4:34:36.979611 elapsed
417000: 4:54:31.467037 elapsed
418000: 5:10:22.826942 elapsed
419000: 5:27:22.085486 elapsed


In [9]:
with open(data_path + "images_saved_400.json", "w") as write_file:
    json.dump(images_saved, write_file)   
            
with open(data_path + "met_objects_400.json", "w") as write_file:
    json.dump(met_objects, write_file)  

In [10]:
len(images_saved)

9716